In [1]:
import numpy as np
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from pyproj import Geod

In [4]:
# load land mask

land = gpd.read_file("../data/ne_10m_land/ne_10m_land.shp")
land_union = land.union_all

In [ ]:
geod = Geod(ellps="WGS84")

In [ ]:
def nearest_land_distance(lat, lon):
    """Return geodesic distance (meters) to nearest land for one point."""
    pt = Point(lon, lat)

    # Query rtree for nearest candidate polygons
    # (reduces search to a handful rather than entire dataset)
    candidate_ids = list(idx.nearest(pt.bounds, 5))

    # Find the nearest point on land among candidates
    nearest_dist = np.inf

    for cid in candidate_ids:
        geom = land.geometry.iloc[cid]
        # nearest point on land polygon to our point
        nearest_pt = nearest_points(pt, geom)[1]
        # geodesic distance
        _, _, d = geod.inv(lon, lat, nearest_pt.x, nearest_pt.y)
        if d < nearest_dist:
            nearest_dist = d

    return nearest_dist